In [3]:
%run ./../../utils/commonImports.py
%matplotlib inline

# Load data

In [4]:
labeled = read_tsv("D:\\Dropbox\\My work\\krypl-project\\dataLabeled\\return-01-loss-max-01-within-day.tsv")
labeled = labeled[['timestamp', 'label']]

In [5]:
data_raw = read_tsv("D:\\Dropbox\\My work\\krypl-project\\analysis\\sg-filter.tsv")
data = data_raw.merge(labeled, on='timestamp', how='inner')
assert data_raw.shape[0] == labeled.shape[0]
assert data.shape[0] == labeled.shape[0]

# Features

In [6]:
from scipy import stats

def lag(arr, l):
    laged = arr[:arr.shape[0]-l]
    laged = np.array([0]*l + laged.tolist())
    return laged

def zscore(x, window):
    r = x.rolling(window=window)
    m = r.mean().shift(1)
    s = r.std(ddof=0).shift(1)
    z = (x-m)/s
    return z


def candle_shape_features(df, _open, close, high, low, lag_len, name_prefix=''):
    df[f'{name_prefix}close_high_ratio_zscore288'] = zscore(close / high, 288)
    df[f'{name_prefix}open_high_ratio_zscore288'] = zscore(_open / high, 288)
    df[f'{name_prefix}low_high_ratio_zscore288'] = zscore(low / high, 288)
    df[f'{name_prefix}open_close_ratio_zscore288'] = zscore((_open - close) / close, 288)
    
    cols = [f'{name_prefix}close_high_ratio_zscore288', 
            f'{name_prefix}open_high_ratio_zscore288', 
            f'{name_prefix}low_high_ratio_zscore288', 
            f'{name_prefix}open_close_ratio_zscore288']
    for col in cols:
        for l in range(1, lag_len):
            df[f"{col}_lag_{l}"] = lag(df[col], l)
    
    return df

In [7]:
cols = ['timestamp', 'period', 'open', 'high', 'low', 'close', 'volume',
       'quoteVolume', 'weightedAverage', 'date', 'label']
featured = data[cols].copy()
featured = candle_shape_features(
    df = featured,
    _open = featured['open'],
    close = featured['close'],
    high = featured['high'],
    low = featured['low'],
    lag_len = 10)
write_tsv(featured, 'D:\\Dropbox\\My work\\krypl-project\\dataFeatures\\candle-shape.tsv')

In [9]:
data.columns

Index(['timestamp', 'period', 'open', 'high', 'low', 'close', 'volume',
       'quoteVolume', 'weightedAverage', 'date', 'close_w3_deg1',
       'open_w3_deg1', 'high_w3_deg1', 'low_w3_deg1', 'close_w3_deg2',
       'open_w3_deg2', 'high_w3_deg2', 'low_w3_deg2', 'close_w3_deg3',
       'open_w3_deg3', 'high_w3_deg3', 'low_w3_deg3', 'close_w3_deg4',
       'open_w3_deg4', 'high_w3_deg4', 'low_w3_deg4', 'close_w5_deg1',
       'open_w5_deg1', 'high_w5_deg1', 'low_w5_deg1', 'close_w5_deg2',
       'open_w5_deg2', 'high_w5_deg2', 'low_w5_deg2', 'close_w5_deg3',
       'open_w5_deg3', 'high_w5_deg3', 'low_w5_deg3', 'close_w5_deg4',
       'open_w5_deg4', 'high_w5_deg4', 'low_w5_deg4', 'close_w10_deg1',
       'open_w10_deg1', 'high_w10_deg1', 'low_w10_deg1', 'close_w10_deg2',
       'open_w10_deg2', 'high_w10_deg2', 'low_w10_deg2', 'close_w10_deg3',
       'open_w10_deg3', 'high_w10_deg3', 'low_w10_deg3', 'close_w10_deg4',
       'open_w10_deg4', 'high_w10_deg4', 'low_w10_deg4', 'label'],
 

In [10]:
cols = ['timestamp', 'period', 'open', 'high', 'low', 'close', 'volume', 'quoteVolume', 'weightedAverage', 
        'date', 'label', 'open_w10_deg1', 'high_w10_deg1', 'low_w10_deg1', 'close_w10_deg1']
featured = data[cols].copy()
featured = candle_shape_features(
    df = featured,
    _open = featured['open_w10_deg1'],
    close = featured['close_w10_deg1'],
    high = featured['high_w10_deg1'],
    low = featured['low_w10_deg1'],
    lag_len = 10)
write_tsv(featured, 'D:\\Dropbox\\My work\\krypl-project\\dataFeatures\\candle-shape-sg.tsv')